# Guided Capstone Step 4 - Pre-processing and Training Data Development

### **The Data Science Method**  


1.   Problem Identification 

2.   Data Wrangling 
  
3.   Exploratory Data Analysis 
 
4.   **Pre-processing and Training Data Development**

5.  Modeling
  * Fit Models with Training Data Set
  * Review Model Outcomes — Iterate over additional models as needed.
  * Identify the Final Model

6.   Documentation
  * Review the Results
  * Present and share your findings - storytelling
  * Finalize Code 
  * Finalize Documentation

In this step of the guided capstone, you'll revisit some of the steps you completed to solve questions in step 4. Revisiting these steps will help you get additional practice completing this kind of work.

**<font color='teal'> Start by loading the necessary packages, as you've done in the previous steps. Print out your current working directory to confirm that you are in the correct project directory. </font>**

In [1]:
#load python packages
import os
import pandas as pd
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

print(os.getcwd())

/Users/KennethMartin1996/Desktop/Springboard/GuidedCapstone


**<font color='teal'>  Load the csv file you created in step 3. Remember, it should be saved inside your data subfolder. Print the first five rows.</font>**

In [2]:
df = pd.read_csv('data/step3_output.csv')
df.head()

,Unnamed: 0,Name,state,summit_elev,vertical_drop,base_elev,trams,fastEight,fastSixes,fastQuads,...,SkiableTerrain_ac,Snow Making_ac,daysOpenLastYear,yearsOpen,averageSnowfall,AdultWeekday,AdultWeekend,projectedDaysOpen,NightSkiing_ac,kmeans_cluster
0,0,Alyeska Resort,Alaska,3939,2500,250,1,0.0,0,2,...,1610.0,113.0,150.0,60.0,669.0,65.0,85.0,150.0,550.0,0
1,1,Eaglecrest Ski Area,Alaska,2600,1540,1200,0,0.0,0,0,...,640.0,60.0,45.0,44.0,350.0,47.0,53.0,90.0,0.0,0
2,2,Hilltop Ski Area,Alaska,2090,294,1796,0,0.0,0,0,...,30.0,30.0,150.0,36.0,69.0,30.0,34.0,152.0,30.0,0
3,3,Arizona Snowbowl,Arizona,11500,2300,9200,0,0.0,1,0,...,777.0,104.0,122.0,81.0,260.0,89.0,89.0,122.0,0.0,1
4,4,Sunrise Park Resort,Arizona,11100,1800,9200,0,0.0,0,1,...,800.0,80.0,115.0,49.0,250.0,74.0,78.0,104.0,80.0,1


In [3]:
df.drop(['Unnamed: 0'], axis=1, inplace=True)

## Create dummy features for categorical variables

**<font color='teal'> Create dummy variables for `State`. Add the dummies back to the dataframe and remove the original column for `State`. </font>**

In [4]:
dummy_vars = pd.get_dummies(df.state)
df_states = pd.concat([df, dummy_vars], axis=1)
df_states.head()

,Name,state,summit_elev,vertical_drop,base_elev,trams,fastEight,fastSixes,fastQuads,quad,...,Rhode Island,South Dakota,Tennessee,Utah,Vermont,Virginia,Washington,West Virginia,Wisconsin,Wyoming
0,Alyeska Resort,Alaska,3939,2500,250,1,0.0,0,2,2,...,0,0,0,0,0,0,0,0,0,0
1,Eaglecrest Ski Area,Alaska,2600,1540,1200,0,0.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Hilltop Ski Area,Alaska,2090,294,1796,0,0.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Arizona Snowbowl,Arizona,11500,2300,9200,0,0.0,1,0,2,...,0,0,0,0,0,0,0,0,0,0
4,Sunrise Park Resort,Arizona,11100,1800,9200,0,0.0,0,1,2,...,0,0,0,0,0,0,0,0,0,0


In [5]:
df_states.drop(['state'], axis=1, inplace=True)

In [6]:
df_states.head()

,Name,summit_elev,vertical_drop,base_elev,trams,fastEight,fastSixes,fastQuads,quad,triple,...,Rhode Island,South Dakota,Tennessee,Utah,Vermont,Virginia,Washington,West Virginia,Wisconsin,Wyoming
0,Alyeska Resort,3939,2500,250,1,0.0,0,2,2,0,...,0,0,0,0,0,0,0,0,0,0
1,Eaglecrest Ski Area,2600,1540,1200,0,0.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Hilltop Ski Area,2090,294,1796,0,0.0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,Arizona Snowbowl,11500,2300,9200,0,0.0,1,0,2,2,...,0,0,0,0,0,0,0,0,0,0
4,Sunrise Park Resort,11100,1800,9200,0,0.0,0,1,2,3,...,0,0,0,0,0,0,0,0,0,0


## Standardize the magnitude of numeric features

**<font color='teal'> Using sklearn preprocessing, standardize the scale of the features of the dataframe except the name of the resort, which you don't need in the dataframe for modeling so it can be droppped here as well. You should  hold out your response variable(s) so that you can have their true values available for model performance review. Let's set `AdultWeekend` to the y variable as your response for scaling and modeling. Later, you will go back and consider the `AdultWeekday`, `dayOpenLastYear`, and `projectedDaysOpen`. For now leave, them in the development dataframe. </font>**

In [7]:
from sklearn import preprocessing
X=df_states.drop(['Name','AdultWeekend'], axis=1)
y=df_states.AdultWeekend
scaler = preprocessing.StandardScaler().fit(X)
X_scaled=scaler.transform(X) 

## Split into training and testing datasets

**<font color='teal'> Using sklearn model selectionm import train_test_split, and create a 75/25 split with the y = `AdultWeekend`. You will start by using the adult weekend ticket price as your response variable for modeling.</font>**

In [8]:
from sklearn.model_selection import train_test_split
y=y.ravel()
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=.25, random_state=1)

Here, you'll start the actual modeling work. First let's fit a multiple linear regression model to predict the `AdultWeekend` price.

# Guided Capstone Step 5 - Modeling

You've cleaned and prepared the datasets, so now it's time to get into the most exciting part of this process: modeling! In this exercise, you will build three different models and compare each model's performance. In the end, you will choose the best model for proving insights to Big Mountain management.  

1.   Problem Identification 

2.   Data Wrangling 
  
3.   Exploratory Data Analysis 
 
4.   Pre-processing and Training Data Development

5.   **Modeling**
  * Fit Models with Training Data Set
  * Review Model Outcomes — Iterate over additional models as needed.
  * Identify the Final Model

6.   Documentation
  * Review the Results
  * Present and share your findings - storytelling
  * Finalize Code 
  * Finalize Documentation

## Fit Models with Training Data Set

**<font color='teal'> Using sklearn, fit the model on your training dataset.</font>**

#### Model 1

In [9]:
#all first model set
from sklearn import linear_model
from sklearn.metrics import explained_variance_score,mean_absolute_error
lm = linear_model.LinearRegression()
model = lm.fit(X_train,y_train)

**<font color='teal'> Predict on the testing dataset and score the model performance with the y_test set and the y-pred values. The explained variance is a measure of the variation explained by the model. This is also known as the R-squared value. </font>**

In [10]:
y_pred = model.predict(X_test)
print(explained_variance_score(y_test, y_pred))
print(mean_absolute_error(y_test, y_pred))

0.9309189536138693
5.153870431915364


## Review Model Outcomes — Iterate over additional models as needed.

In [11]:
rm = linear_model.Ridge()
ridge_model = rm.fit(X_train,y_train)
y_pred_ridge = ridge_model.predict(X_test)
print(explained_variance_score(y_test, y_pred_ridge))
print(mean_absolute_error(y_test, y_pred_ridge))

0.9376314046324954
4.996784063769456


In [12]:
lasso = linear_model.Lasso()
lasso_model = lasso.fit(X_train,y_train)
y_pred_lasso = lasso_model.predict(X_test)
print(explained_variance_score(y_test, y_pred_lasso))
print(mean_absolute_error(y_test, y_pred_lasso))

0.9188188417057608
5.689218914883504


**<font color='teal'> Print the intercept value from the linear model. </font>**

In [13]:
print(model.intercept_)

64.12571363041931


**<font color='teal'> The intercept is the mean `AdultWeekend` price for all the resorts given the other characteristics. The addition or subtraction of each of the coefficient values in the regression are numeric adjustments applied to the intercept to provide a particular observation's value for the `AdultWeekend` resulting value. Also, because you took the time to scale your x values in the training data, you can compare each of the coefficients for the features to determine the feature's importance. Print the coefficient values from the linear model and sort in descending order to identify the top ten most important features.</font>** 

Hint: make sure to review the absolute value of the coefficients, because the adjustment may be positive or negative, but what you are looking for is the magnitude of impact on your response variable.

In [14]:
print(model.coef_)

[-1.84818542e+01  5.87154474e+00  1.78772890e+01 -1.83503819e+12
 -1.80402230e+11 -2.13568307e+12 -7.20418050e+12 -4.30044756e+12
 -5.30616162e+12 -5.94815359e+12 -6.74977690e+12  1.90032647e+13
 -1.23666382e+00 -1.90071106e-01 -4.68139648e-01 -1.68945312e-01
  8.98635864e-01  3.26965332e-01  3.77990723e-01 -2.04101562e-01
  1.92138672e+01 -1.06201172e-01  7.35229492e-01 -5.15136719e-01
 -2.14610987e+12 -1.75496867e+12 -5.51958384e+12 -5.64032522e+12
 -2.76213010e+12 -4.23274037e+12 -2.47432215e+12 -1.75496867e+12
 -2.14610987e+12 -3.68291095e+12 -1.24284050e+12 -4.05890729e+12
 -6.40176450e+12 -4.55748247e+12 -1.75496867e+12 -4.23274037e+12
 -2.47432215e+12 -4.85671101e+12 -1.75496867e+12 -3.68291095e+12
 -6.78348145e+12 -3.02110332e+12 -2.76213010e+12 -3.87607736e+12
 -5.26713491e+12 -1.24284050e+12 -1.75496867e+12 -1.24284050e+12
 -4.39864340e+12 -4.70997231e+12 -2.47432215e+12 -3.87607736e+12
 -2.47432215e+12 -4.85671101e+12 -3.47768608e+12]


In [15]:
coef_dict = dict(zip(X.columns, abs(model.coef_)))
coef_df = pd.DataFrame(coef_dict.items())
desc_df = coef_df.sort_values([1], ascending=False)
desc_df.head(10)

,0,1
11,total_chairs,1.900326e+13
6,fastQuads,7.204180e+12
44,New York,6.783481e+12
10,surface,6.749777e+12
36,Michigan,6.401765e+12
9,double,5.948154e+12
27,Colorado,5.640325e+12
26,California,5.519584e+12
8,triple,5.306162e+12
48,Pennsylvania,5.267135e+12


**<font color='teal'>You should see that the top ten important features are different states. However, the state is not something the managers at the Big Mountain Resort can do anything about. Given that you care more about actionable traits associated with ticket pricing, rebuild the model without the state features and compare the results. </font>**

#### Model 2

In [16]:
X=df.drop(['Name','AdultWeekend','state'], axis=1)
y=df.AdultWeekend
scaler = preprocessing.StandardScaler().fit(X)
X_scaled=scaler.transform(X) 

y=y.ravel()
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=.25, random_state=1)

In [17]:
model2 = lm.fit(X_train,y_train)
y_pred = model2.predict(X_test)
print(explained_variance_score(y_test, y_pred))
print(mean_absolute_error(y_test, y_pred))
print(model2.intercept_)

0.9237016967243137
5.522988250727305
64.09002492151635


In [18]:
coef_dict = dict(zip(X.columns, abs(model.coef_)))
coef_df = pd.DataFrame(coef_dict.items())
desc_df = coef_df.sort_values([1], ascending=False)
desc_df.head(10)

,0,1
20,AdultWeekday,19.904636
0,summit_elev,13.341084
2,base_elev,11.305804
1,vertical_drop,4.957778
12,Runs,1.639027
7,quad,1.534423
19,averageSnowfall,1.435398
8,triple,1.412285
23,kmeans_cluster,1.371048
10,surface,1.329273


**<font color='teal'> When reviewing your new model coefficients, note that `summit_elev` is now in the number two spot. From a managerial perspective, this is also difficult to change and highly correlated with `base_elev` and `vertical_drop`.  This time rebuild the model without the state features and without the `summit_elev` and without `base_elev`and compare the results. </font>**

#### Model 3

In [19]:
X=df.drop(['Name','AdultWeekend','state','summit_elev','base_elev'], axis=1)
y=df.AdultWeekend
scaler = preprocessing.StandardScaler().fit(X)
X_scaled=scaler.transform(X) 

y=y.ravel()
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=.25, random_state=1)

In [20]:
model3 = lm.fit(X_train,y_train)
y_pred = model3.predict(X_test)
print(explained_variance_score(y_test, y_pred))
print(mean_absolute_error(y_test, y_pred))
print(model3.intercept_)

0.930026472353041
5.326744869611585
64.09125212896649


In [21]:
coef_dict = dict(zip(X.columns, (abs(model.coef_))))
coef_df = pd.DataFrame(coef_dict.items())
desc_df = coef_df.sort_values([1], ascending=False)
desc_df.head(10)

,0,1
18,AdultWeekday,19.888352
0,vertical_drop,1.755140
10,Runs,1.727629
5,quad,1.487769
6,triple,1.396678
17,averageSnowfall,1.392821
21,kmeans_cluster,1.316956
8,surface,1.269381
15,daysOpenLastYear,0.936905
4,fastQuads,0.765670


## Identify the Final Model

**<font color='teal'> Review the model performances in the table below and choose the best model for proving insights to Big Mountain management about what features are driving ski resort lift ticket prices. Type your choice in the final markdown cell. You will explain your selection during the next step of the guided casptone. </font>**

| Model | Explained Variance| Mean Absolute Error|Features Dropped|
| --- | --- | --- | --- |
| Model 1. | 0.9309 | 5.154 |-|
| Model 2. | 0.9237| 5.523 |'state'|
| Model 3. | 0.9300 | 5.327 |'state','summit_elev','base_elev'|

Model Selection:

Model 1 has the highest Explained Variance and the lowest Mean Absolute Error. However, since Big Mountain has no control over state, summit elevation, or base elevation and the Explained Variance and Mean Absolute Error of Model 3 are very similar to that of Model 1, Model 3 is likely the most helpful model for providing insights to management.